### Commands

In [1]:
from firecloud import api as firecloud_api
import fc_interface

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
# res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN (all batches)
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch (no samples from other batches)
batches_info = pd.read_excel(paths_to_samples_info)
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [ ]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

In [ ]:
### Create PoN from all normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [6]:
### Create PoN with normals from batch + other normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [15]:
# batch_id = 'TSCA21'
# batch_path = batches_info.loc[batches_info.tsca_id==batch_id, 'path_to_samples_info'].item()
# pd.read_table(batch_path)

,sample_id,individual_id,clean_bam_file_capture,external_id_validation,aggregation_product_name_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,picard_aggregation_type_validation,processed_subtype_validation,source_subtype_validation,squid_sample_id_validation,tumor_subtype,short_letter_code
0,AA53-Tumor-SM-GBJ9E,AA53,/xchip/clf/seq_data/processed_for_fh/tsca21_20...,AA53T_2_OPAC_p11,TSCA Rapid Cancer Detection Panel v2,SM-GBJ9E,SM-GB4TR,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",AA53T_2_OPAC_p11,Metastatic,TM
1,AA55-Tumor-SM-GBJBF,AA55,/xchip/clf/seq_data/processed_for_fh/tsca21_20...,AA55T_CM_p12,TSCA Rapid Cancer Detection Panel v2,SM-GBJBF,SM-GB4DD,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",AA55T_CM_p12,Metastatic,TM
2,AA56-Tumor-SM-GBJ92,AA56,/xchip/clf/seq_data/processed_for_fh/tsca21_20...,AA56T_OPAC_p12,TSCA Rapid Cancer Detection Panel v2,SM-GBJ92,SM-GB4U1,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",AA56T_OPAC_p12,Metastatic,TM
3,AB059-Tumor-SM-GBJAS,AB059,/xchip/clf/seq_data/processed_for_fh/tsca21_20...,AB059T_CM_p12,TSCA Rapid Cancer Detection Panel v2,SM-GBJAS,SM-GB4U5,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",AB059T_CM_p12,Metastatic,TM
4,AB059-Tumor-SM-GBJ9G,AB059,/xchip/clf/seq_data/processed_for_fh/tsca21_20...,AB059T_OPAC_p12,TSCA Rapid Cancer Detection Panel v2,SM-GBJ9G,SM-GB4TZ,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",AB059T_OPAC_p12,Metastatic,TM
5,BT1016-Tumor-SM-GBJ9Q,BT1016,/xchip/clf/seq_data/processed_for_fh/tsca21_20...,BT1018_NSA_p5_LowAdherent,TSCA Rapid Cancer Detection Panel v2,SM-GBJ9Q,SM-GB4U4,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",BT1018_NSA_p5_LowAdherent,Primary,TP
6,BT1160-Tumor-SM-GBJBE,BT1160,/xchip/clf/seq_data/processed_for_fh/tsca21_20...,BT1160_KL_CL,TSCA Rapid Cancer Detection Panel v2,SM-GBJBE,SM-G8SIU,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",BT1160_KL_CL,Primary,TP
7,BT1160-Tumor-SM-GBJ9C,BT1160,/xchip/clf/seq_data/processed_for_fh/tsca21_20...,BT1160_KL_Primary,TSCA Rapid Cancer Detection Panel v2,SM-GBJ9C,SM-G8SIO,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",BT1160_KL_Primary,Primary,TP
8,BT869-Tumor-SM-GBJ9O,BT869,/xchip/clf/seq_data/processed_for_fh/tsca21_20...,BT869_KL_CL,TSCA Rapid Cancer Detection Panel v2,SM-GBJ9O,SM-G8SIP,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",BT869_KL_CL,Primary,TP
9,CCLF_AA1016-Tumor-SM-GBJAG,CCLF_AA1016,/xchip/clf/seq_data/processed_for_fh/tsca21_20...,CCLF_AA1016T_ASC_OPAC_p3,TSCA Rapid Cancer Detection Panel v2,SM-GBJAG,SM-GB4CK,Tumor,PCR,DNA:DNA Somatic,Tissue:Fresh Tissue,CCLF_AA1016T_ASC_OPAC_p3,Metastatic,TM


In [ ]:
### Update WDLs
fc_interface.download_remote_wdls(namespace, workspace)

---

---